In [1]:
traindata_file = "traindata_x_2.json"
model="vi_core_news_lg"
output_dir='/output'
prefix_name = "category_x"
n_iter=30
n_data =5000


In [2]:
import os
import json
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import random


In [22]:
def load_data(file):
    with open(traindata_file, 'r', encoding="utf8") as f:
        return [json.loads(line) for line in f.readlines()]

In [23]:
data = load_data(traindata_file)[0]

In [24]:
data

{'content': '15/05/2021\nMở Bán DỰ ÁN NHÀ Ở XÃ HỘI Thượng Thanh, Long Biên giá từ 16 triệu Miên phí HỒ S\nLoại tin rao Bán căn hộ chung cư\nHà Nội\nGia Quất\nLong Biên\nTên liên hệ Nguyễn Phượng\nGiá 580 Triệu\nGia Quất - Phường Ngọc Lâm -  Long Biên -  Hà Nội\nDiện tích 37\n1\nMặt tiền 4\n0981922984',
 'annotation': [{'label': ['date'],
   'points': [{'start': 0, 'end': 10, 'text': '15/05/2021'}]},
  {'label': ['title'],
   'points': [{'start': 11,
     'end': 90,
     'text': 'Mở Bán DỰ ÁN NHÀ Ở XÃ HỘI Thượng Thanh, Long Biên giá từ 16 triệu Miên phí HỒ S'}]},
  {'label': ['category'],
   'points': [{'start': 91,
     'end': 123,
     'text': 'Loại tin rao Bán căn hộ chung cư'}]},
  {'label': ['region'],
   'points': [{'start': 124, 'end': 131, 'text': 'Hà Nội'}]},
  {'label': ['street'],
   'points': [{'start': 132, 'end': 140, 'text': 'Gia Quất'}]},
  {'label': ['district'],
   'points': [{'start': 141, 'end': 150, 'text': 'Long Biên'}]},
  {'label': ['user'],
   'points': [{'st

In [30]:

def convert_data(traindata_file):
    try:
        training_data = []
        lines = load_data(traindata_file)[0:1000]
        for line in lines:
            data = line

            text = data['content']
            entities = []
            if data['annotation'] is not None:
                for annotation in data['annotation']:
                    # only a single point in text annotation.
                    point = annotation['points'][0]
                    labels = annotation['label']
                    # handle both list of labels or a single label.
                    if not isinstance(labels, list):
                        labels = [labels]

                    for label in labels:
                        # dataturks indices are both inclusive [start, end]
                        # but spacy is not [start, end)
                        check = True
#                         for s, e, l in entities:
#                             if not (point['start'] < point['end'] < s < e or point['start'] > point['end'] > s > e):
#                                 check = False
#                                 break
                                
                        if check and label in ["user"]:      
                            entities.append((
                                point['start'],
                                point['end'], # + 1,
                                label
                            ))

            training_data.append((text, {"
                                          ": entities}))
        return training_data
    except Exception:
#         logging.exception("Unable to process " + traindata_file)
        return None

SPACY_DATA = convert_data(traindata_file)
print(SPACY_DATA[0:10])

[('15/05/2021\nMở Bán DỰ ÁN NHÀ Ở XÃ HỘI Thượng Thanh, Long Biên giá từ 16 triệu Miên phí HỒ S\nLoại tin rao Bán căn hộ chung cư\nHà Nội\nGia Quất\nLong Biên\nTên liên hệ Nguyễn Phượng\nGiá 580 Triệu\nGia Quất - Phường Ngọc Lâm -  Long Biên -  Hà Nội\nDiện tích 37\n1\nMặt tiền 4\n0981922984', {'entities': [(151, 176, 'user')]}), ('24/12/2020\nSiêu phẩm-Nhà Cổ Nhuế-Ngõ rộng-xây mới-DT34m2-5 Tầng-2.65 TỶ\nLoại tin rao Bán căn hộ chung cư\nHà Nội\nCổ Nhuế\nBắc Từ Liêm\nTên liên hệ Nguyễn Hữu Phong\nGiá 2.65 Tỷ\nCổ Nhuế - Phường Cổ Nhuế 2 -  Bắc Từ Liêm -  Hà Nội\n0984897688', {'entities': [(134, 162, 'user')]}), ('hôm nay\nCăn hộ 5* KS La Vela Saigon 57m2,2 PN,Q3,bay nhanh 6.3 tỷ\nLoại tin rao Bán căn hộ chung cư\nHồ Chí Minh\nTên liên hệ Nguyễn Tô Quyên\nGiá 6,3 Tỷ\nDiện tích 57\n3\nMặt tiền 7\n0912978184', {'entities': [(111, 138, 'user')]}), ('hôm nay\nCần bán nhanh căn hộ CC bình vượng 200 quang trung, SĐCC 112 m2, 15,2 triệu/M2\nLoại tin rao Bán căn hộ chung cư\nHà Nội\nTên li

In [31]:

from spacy.training import Example
from spacy.util import minibatch

TRAIN_DATA = SPACY_DATA[0:n_data]

nlp = spacy.blank("vi")
# nlp = spacy.load('model')

examples = []
for text, annots in TRAIN_DATA:
    doc = nlp.make_doc(text)
#     print(doc)
    examples.append(Example.from_dict(doc, annots))
    
if "ner" not in nlp.pipe_names:
    nlp.add_pipe("ner", last=True)

# ner = nlp.get_pipe("ner")
# for _, annotations in TRAIN_DATA:
#     for ent in annotations.get('entities'):
#         ner.add_label(ent[2])
# move_names = list(ner.move_names)
# print(move_names) 

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]


        


C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "15/05/2021
Mở Bán DỰ_ÁN NHÀ Ở XÃ_HỘI Thượng_Thanh,..." with entities "[(151, 176, 'user')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "24/12/2020
Siêu_phẩm-Nhà_Cổ_Nhuế-Ngõ rộng-xây mới-..." with entities "[(134, 162, 'user')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "hôm_nay
Cần bán nhanh căn_hộ CC bình

C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "hôm_nay
NHÀ ĐẸP HÀNG ĐIẾU – 60M2 X 5 TẦNG – KINH_D..." with entities "[(106, 134, 'user')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "hôm_nay
Bán nhà phố Phan_Kế Bính, diện_tích 57m2, ..." with entities "[(143, 170, 'user')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "hôm_nay
Bán Gấp toà nhà làm Homestay

C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "hôm_nay
Bán Nhà_Phân lô Gara Thang_máy Phố Nguyễn_..." with entities "[(168, 191, 'user')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "hôm_nay
Biệt_Thự Sân Vườn_Xuân_Đỉnh ô_tô tránh 3 t..." with entities "[(136, 160, 'user')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "hôm_nay
Bán Nhà_Phân lô Gara Thang_m

C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "28/01/2021
Mặt_phố Lý_Nam_Đế - Hoàn_Kiếm - Đoạn đẹ..." with entities "[(146, 173, 'user')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "28/01/2021
2 MẶT TIỀN MẶT_PHỐ ĐẠI LA, HAI BÀ TRƯNG..." with entities "[(136, 163, 'user')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "28/01/2021
LÔ GÓC MẶT_PHỐ LÊ THANH N

C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "hôm_nay
HIẾM CÓ KHÓ TÌM, DƯỚI 2 TỶ CÓ NHÀ BA ĐÌNH,..." with entities "[(109, 127, 'user')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "hôm_nay
ĐỘI CẤN, 35M2, NHÀ THOÁNG, MẶT_TIỀN RỘNG, ..." with entities "[(128, 146, 'user')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "hôm_nay
Bán gấp nhà TT quận Ba_Đình 

C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "hôm_nay
ĐÀO TẤN, 3 TỶ, 30M2, CHỦ NHÀ CẦN BÁN TRƯỚC..." with entities "[(106, 124, 'user')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "hôm_qua
GIÁ RẺ - BÁN ĐẤT TẶNG NHÀ Ngọc_Khánh DT 90..." with entities "[(122, 152, 'user')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "hôm_qua
Nhà đẹp phố Núi_Trúc, Ba Đìn

C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "hôm_nay
Bán nhà Xuân_Diệu, View_Hồ, ÔTÔ, 4.2 tỷ.
L..." with entities "[(101, 127, 'user')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "hôm_nay
Bán nhà Yên_Hoa, Ô_tô, View_Hồ, nhà đẹp Ở ..." with entities "[(106, 132, 'user')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "hôm_nay
Bán nhà Yên_Phụ, hai thoáng,

C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "14/05/2021
Bán nhà 2 tầng 134m² phố Đặng_Thanh_Mai..." with entities "[(145, 173, 'user')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "29/11/2020
HIẾM- 3.2 tỷ nhà đẹp 48m2 4 tầng Thụy_K..." with entities "[(125, 151, 'user')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "hôm_nay
KHÔNG CÓ CĂN THỨ 2, THỤY KHU

C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "05/05/2021
6 tầng thang_máy Tôn_Đức_Thắng, nội_thấ..." with entities "[(135, 159, 'user')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "hôm_nay
Bán gấp nhà TT Đống_Đa, 3 mặt ngõ, 4 mặt t..." with entities "[(123, 149, 'user')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "hôm_nay
Siêu phẩm Thổ_Quan 3 thoáng 

C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "09/05/2021
Bán nhà đẹp sát phố Tây_Sơn, trung_tâm ..." with entities "[(130, 151, 'user')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "07/05/2021
VIEW HỒ - Ô_TÔ ĐỖ CỬA - KINH_DOANH, VĂN..." with entities "[(108, 133, 'user')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "05/05/2021
HÀNG HIẾM NGÃ TƯ SỞ 45M2X

In [32]:
with nlp.disable_pipes(*other_pipes):
    nlp.initialize(lambda: examples)
    for i in range(n_iter):
        print("Iter ", i)
        random.shuffle(examples)
        losses={}
        for batch in minibatch(examples, size=8):
            nlp.update(batch,losses=losses)
        print("Losses: ", losses)
        

Iter  0
Losses:  {'ner': 3254.4829586372607}
Iter  1
Losses:  {'ner': 42.240727935998244}
Iter  2


KeyboardInterrupt: 

In [ ]:
# nlp.to_disk("./en_example_pipeline")
# test_text = """BÁN GẤP NHÀ PHỐ ĐÀO TẤN, DIỆN TÍCH 50M2, XÂY 4 TẦNG, MẶT TIỀN 5.5M, GIÁ 5.3 Khu Vực: Bán Nhà Riêng Tại Đường Đào Tấn - Phường Cống Vị - Ba Đình - Hà Nội Giá: 5,3 Tỷ  Diện tích: 50 m² hôm nay Thông tin mô tả + Nhà nằm trong ngõ 12 Đào Tấn ô tô tránh có vỉa hè, ngõ thông nhiều phố, nhà cách ô tô đúng 20m, nhà vẫn còn rất mới, khách mua về là vào ở được ngay. + Nhà mới xây được mấy năm nhưng còn rất chắc chắn. Thiết kế nhà bê tông cốt thép chắc chắn, thiết kế hiện đại, nhà bao gồm: phòng khách, bếp, 04 phòng ngủ, phòng thờ, sân phơi + Sổ đỏ chính chủ, pháp lý rõ ràng, sẵn sang giao dịch + Giá 5.3 tỷ, có thương lượng cho khách có thiện trí Liên Hệ: Thanh Tùng BĐS Ba Đình: 0912142902, Quý khách gọi ngay để được tư vấn nhiệt tình và xem nhà miễn phí Cần Bán, ở ngay, ngõ thoáng, nhà còn mới, Ngõ thông, Đào Tấn, Ba Đình Liên hệ: 09121*** hiện số Loại tin rao Bán nhà riêng Pháp lý Sổ Hồng Số phòng 0 Đường vào 0 m Mặt tiền 0 m Số tầng 4 Số toilet 0 Thông tin liên hệ Tên liên hệ: Phan Thanh Tùng Loại tin: Môi giới Địa chỉ: Ba Đình Tỉnh thành Hà Nội Điện thoại: 0912142902 Email: thanhtungks68@gmail.com"""
test_text = """'Loại tin Bán nhà riêng Tên liên hệ: Phan Thanh Tùng Giá 4,7 Tỷ Điện tích 38 m² Số tầng 5 Số phòng 0 hôm nay 0912142902 Mặt tiền 0 m'"""
doc = nlp(test_text)
for ent in doc.ents:
    print(" > ", ent.label_, ent.text)


In [63]:
import random

def train_spacy(TRAIN_DATA, iterations):

    #Create the blank spacy model
    nlp = spacy.blank("en")
    
    #add the ner component to the pipeline if it's not there
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    
    #add all labels to the spaCy model
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    
    #eliminate the effect of the training on other pipes and 
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    
    #begin training
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print ("Starting iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                            [text],
                            [annotations],
                            drop=0.2,
                            sgd=optimizer,
                            losses=losses
                )
            print
    return (nlp)

#run function and create a trained model
trained_nlp = train_spacy(TRAIN_DATA, 10)

NameError: name 'TRAIN_DATA' is not defined

In [72]:
nlp = spacy.blank("vi")
def create_spacy_data(data): 
    docbin = DocBin()
    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span:
                ents.append(span)
            else:
                print("skip")
        doc.ents = ents
        docbin.add(doc)
    return docbin
split_point = int(len(SPACY_DATA)*0.9)
train_data = create_spacy_data(SPACY_DATA[0:split_point])
train_data.to_disk("./data/train_data.spacy")
valid_data = create_spacy_data(SPACY_DATA[split_point:-1])
valid_data.to_disk("./data/valid_data.spacy")

 21%|████████████████▋                                                              | 207/982 [00:00<00:01, 708.82it/s]

skip
skip
skip
skip
skip
skip
skip
skip


100%|███████████████████████████████████████████████████████████████████████████████| 109/109 [00:00<00:00, 768.05it/s]


skip
skip
skip
skip
skip
